In [1]:
import pandas as pd

In [12]:
df = pd.read_json('../../data/amazon-electronics-reviews.json', lines=True)

In [13]:
df = df[['reviewText']]

In [14]:
df.head(2)

,reviewText
0,We got this GPS for my husband who is an (OTR)...
1,"I'm a professional OTR truck driver, and I bou..."
